In [ ]:
#LDA WITH RANDOM FOREST
import os
import zipfile
import scipy.io as sio
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier  # Import Random Forest classifier
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score



# Specify the path to your zip file
zip_file_path = "index01 (1).zip"

# Specify the directory where you want to extract the files
extracted_dir = "extracted_data"

# Create the extracted directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Extract the files from the zip archive
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# List the files in the extracted directory
extracted_files = os.listdir(extracted_dir)

# Create empty lists to store features and gesture labels
rms_features = []
mav_features = []
variance_features = []
zero_crossing_features = []
wl_features = [] 
gesture_labels = []

# Define the label values you want to classify (1, 3, 5, 7, 11)
target_label_values = [1, 2, 4, 5]

# Process each extracted file
for file_name in extracted_files:
    if file_name.endswith(".mat"):  # Assuming the files are .mat format
        # Load the .mat file
        mat_data = sio.loadmat(os.path.join(extracted_dir, file_name))

        # Access the sEMG data using the correct variable name ('data' based on your previous output)
        sEMG_signal = mat_data['data']

        # Calculate RMS (Root Mean Square) feature
        rms_feature = np.sqrt(np.mean(np.square(sEMG_signal)))

        # Calculate MAV (Mean Absolute Value) feature
        mav_feature = np.mean(np.abs(sEMG_signal))

        # Calculate Variance feature
        variance_feature = np.var(sEMG_signal)

        # Calculate Zero Crossing feature
        zero_crossings = np.where(np.diff(np.sign(sEMG_signal)))[0]
        zero_crossing_feature = len(zero_crossings)

        # Access the gesture labels using the correct variable name ('gesture')
        gesture_label = mat_data['gesture'][0][0]  # Assuming 'gesture' is the correct variable name

        # Check if the gesture label is one of the target label values
        if gesture_label in target_label_values:
            # Append features and gesture label to the respective lists
            rms_features.append(rms_feature)
            mav_features.append(mav_feature)
            variance_features.append(variance_feature)
            zero_crossing_features.append(zero_crossing_feature)
            wl_features.append(wl_feature)
            gesture_labels.append(gesture_label)

# Convert lists to numpy arrays
X = np.column_stack((rms_features, mav_features, variance_features, zero_crossing_features))  # Feature matrix
y = np.array(gesture_labels)  # Target labels

# Encode the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# Perform Linear Discriminant Analysis (LDA) for feature selection
lda = LDA()
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

# Create a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the number of trees

# Train the classifier on the training data with LDA-selected features
clf.fit(X_train_lda, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_lda)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
#LDA WITH KNN
from sklearn.neighbors import KNeighborsClassifier  # Import KNN classifier
# Create a K-Nearest Neighbors (KNN) classifier
clf = KNeighborsClassifier(n_neighbors=3)  # You can adjust the number of neighbors

# Train the classifier on the training data with LDA-selected features
clf.fit(X_train_lda, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_lda)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
#LDA WITH SVM
# Create a Support Vector Machine (SVM) classifier
from sklearn.svm import SVC

clf = SVC(kernel='linear', C=1.0, random_state=30)

# Train the classifier on the training data with LDA-selected features
clf.fit(X_train_lda, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_lda)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

